In [1]:
from datasets import load_dataset
# from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from transformers import BertTokenizer, BertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import pandas as pd
import torch
import wandb
torch.cuda.empty_cache()

In [2]:
train_data = pd.read_csv('si630w22-hw3-train.csv')
dev_data = pd.read_csv('si630w22-hw3-dev.csv')
# test_data = pd.read_csv('data\si630w22-hw3-test.public.csv')
q_and_a_data = pd.read_csv('si630w22-hw3-data.csv')

In [3]:
#combine the the different ratings
train_df = train_data.groupby('id').mean()
train_df = train_df.reset_index()
dev_df = dev_data.groupby('id').mean()
dev_df = dev_df.reset_index()

In [4]:
train_df.rename(columns={'id':'question_id'}, inplace = True)
dev_df.rename(columns={'id':'question_id'}, inplace = True)
train_df.rename(columns={'rating':'labels'}, inplace = True)
dev_df.rename(columns={'rating':'labels'}, inplace = True)

q_and_a_data['text'] = q_and_a_data.question_text + '[SEP]' + q_and_a_data.reply_text

merged_train_data = pd.merge(train_df,q_and_a_data[['text','question_id']], on='question_id', how='left')
merged_train_data.dropna(subset=['labels'], inplace = True)
merged_train_data.drop(columns=['question_id'], inplace = True)
merged_train_data.to_csv('merged_train_data.csv', index=False)

merged_dev_data = pd.merge(dev_df, q_and_a_data[['text','question_id']], on='question_id', how='left')
merged_dev_data.dropna(subset=['labels'], inplace = True)
merged_dev_data.drop(columns=['question_id'], inplace = True)
merged_dev_data.to_csv('merged_dev_data.csv', index=False)

# merged_test_data = pd.merge(q_and_a_data, test_data, on='question_id')
# merged_test_data.to_csv('data\merged_test_data.csv', index=False)

In [5]:
data_files = {"train": "merged_train_data.csv",
              "dev": "merged_dev_data.csv",}
#               "test": "data\merged_test_data.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Using custom data configuration default-f947e6f1a3a233ba


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/seffendi/.cache/huggingface/datasets/csv/default-f947e6f1a3a233ba/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 3779
    })
    dev: Dataset({
        features: ['labels', 'text'],
        num_rows: 811
    })
})

In [6]:
# dataset['train']['text']

In [9]:
model = BertForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
# tokenizer = BertTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
tokenizer = BertTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased", padding = True, truncation=True ,max_length =512)

OSError: We couldn't connect to 'https://huggingface.co/' to load this model and it looks like microsoft/MiniLM-L12-H384-uncased is not the path to a directory conaining a a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [71]:
tokenized_dataset= dataset.map(lambda x: tokenizer(x['text'],padding = 'max_length', max_length =512))

  0%|          | 0/3779 [00:00<?, ?ex/s]

  0%|          | 0/811 [00:00<?, ?ex/s]

In [72]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3779
    })
    dev: Dataset({
        features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 811
    })
})

In [73]:
tokenized_dataset['train']

Dataset({
    features: ['labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3779
})

In [74]:
# tokenized_dataset
len(tokenized_dataset['train']['input_ids'][2])
# tokenized_dataset['train'][0]

512

In [104]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask','labels'])

dataloader = torch.utils.data.DataLoader(tokenized_dataset['train'], batch_size=8)
# next(iter(dataloader))
# tokenized_dataset['train'][0]

In [105]:
# training_args = TrainingArguments("test-trainer")
training_args = TrainingArguments(
    output_dir = 'BERTSeq',
    num_train_epochs = 3,
    
#     do_train = True,
#     do_eval = True,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    learning_rate=1e-4,
#     logging_strategy = 'epoch',
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
#     warmup_steps = 250,
#     weight_decay = 0.01,
    seed =0,
#     metric_for_best_model = 'eval_loss',
    load_best_model_at_end = True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [106]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
#     data_collator=data_collator,
#     tokenizer=tokenizer
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [107]:
# labels

In [109]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3779
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2835


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.78 GiB total capacity; 14.36 GiB already allocated; 1.50 MiB free; 14.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [85]:
ii = trainer.train_dataset[1]
# model(**ii, labels =torch.tensor([1]).unsqueeze(0) )
ii

{'labels': tensor(3.7500),
 'input_ids': tensor([  101,  2054,  2003,  1010,  1999,  2115,  5448,  1010,  1996,  6517,
          6155,  2102, 12700, 10457,  7062,  2412,  1029,   102,  2026,  2158,
          2852, 17655, 20160, 18940, 11624, 14503,  3215,  2001,  2141,  2302,
          2010,  3008,  1010,  5102,  2004,  1037, 25781,  2008,  2288,  7383,
          1010,  1998,  2467, 12386,  2000,  1037, 20228,  4017, 22571,  2271,
          2008,  2002,  2071,  4089,  3786,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            

In [86]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"[CLS] is there someone you turned down in the past, and now regret not going out with? why? [SEP] idk if this counts but my when i was younger my crush asked me to hang out with her like just two of us, and i kinda just ghosted her for the day because i was too coward to be with her without anyone else around because i'm an huge introvert and i'm shy af also my social skills are really bad... i honestly regret that a lot but at least next time i won't do the same mistake. hopefully... [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [133]:
pd.options.display.max_colwidth = 500
merged_train_data.text.head(1)

0    Is there someone you turned down in the past, and now regret not going out with? Why?[SEP]Idk if this counts but my when I was younger my crush asked me to hang out with her like just two of us, and I kinda just ghosted her for the day because I was too coward to be with her without anyone else around because I'm an huge introvert and I'm shy af also my social skills are really bad... I honestly regret that a lot but at least next time I won't do the same mistake. Hopefully...
Name: text, dtype: object

In [217]:
trainer.train_dataset[0].keys()

dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])

In [134]:
# tokenizer.decode(trainer.train_dataset[1]["token_type_ids"])

In [52]:
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)
# model(**inputs)
# type(inputs), inputs

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [85]:
labels

tensor([[1]])

In [116]:
# x =tokenized_dataset['train'][1]
type(tokenized_dataset['train'][1]), x
for (k, v) in x.items():
    print(k,v)
tuple(x[k] for k in x.keys())

input_ids tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
         2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
         2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
         2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
         2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
         2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
         1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
          102])
token_type_ids tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
attention_mask tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,


(tensor([  101,  2054,  1005,  1055,  2242,  3835,  2017,  2066,  2000,  2079,
          2074,  2000,  2022,  3835,  1029,   102,  2507, 19394,  2015,  1012,
          2009,  1521,  1055,  5186,  3733,  2000,  2079,  1998,  2071,  2689,
          2619,  1521,  1055,  2154,  1012, 10657,  2022,  6176,  2000,  2079,
          2061,  1999,  1037,  2126,  2008,  2987,  1521,  1056,  3499,  1996,
          2060,  2283,  2000,  2228,  2017,  1521,  2128,  7294,  2006,  2068,
          1010,  2295,  1012,  2061,  2009,  1521,  1055,  1037,  5703,  1012,
           102]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1